In [1]:
import os
os.chdir("../")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ImageEnhancerConfig:
    load_img_low_res: Path
    save_high_res_img: Path
    model_url: str

In [9]:
from Image_enhance.constants import *
from Image_enhance.utils.common import read_yaml,  create_directories
from Image_enhance import logger

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        """
        Initializes ConfigurationManager with provided filepaths.

        Args:
            config_filepath (str): Filepath to configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (str): Filepath to parameters file. Defaults to PARAMS_FILE_PATH.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    
    def get_image_enhancer_config(self) -> ImageEnhancerConfig:
        """
        Retrieves image ingestion configuration settings.

        Returns:
            image_ingestion_config (ImageIngestionConfig): Image ingestion configuration object.
        """
        config = self.config.image_enhance
        
        image_enhancer_config = ImageEnhancerConfig(
            load_img_low_res=config.load_img_low_res,
            save_high_res_img=config.save_high_res_img,
            model_url=self.params.ENHANCE.MODEL_URL
        )

        return image_enhancer_config



In [11]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import imageio
import time

In [12]:
class ImageEnhacer():
    def __init__(self, config:ImageEnhancerConfig):
        self.config=config
    
    def preprocess_image(self):
        """
        Preprocesses an image to make it ready for the model.

        Inputs:
            image_path (str): Path to the image file.

        Outputs:
            tf.Tensor: Processed image tensor ready for model input.
        """

        load_low_res_img_path = self.config.load_img_low_res
        # Read the image file and decode it into a tensor.
        image = tf.image.decode_image(tf.io.read_file(load_low_res_img_path))
        logger.info(f"Image obtained from : {load_low_res_img_path}")

        logger.info(f"Transfroming image into a tensor")
        # If PNG, remove the alpha channel. The model only supports images with 3 color channels.
        if image.shape[-1] == 4:
            image = image[...,:-1]

        # get image dimension  divisible by 4
        image_size = (tf.convert_to_tensor(image.shape[:-1]) // 4) * 4


        # Crop the image to ensure its dimensions are divisible by 4.
        image = tf.image.crop_to_bounding_box(image, 0, 0, image_size[0], image_size[1])
        
        # Convert the image to float32 data type.
        image = tf.cast(image, tf.float32)

        # Expand the dimensions of the image tensor to match the expected input shape of the model.
        logger.info(f"Returning tensor image")
        return tf.expand_dims(image, 0)
    
    def low_to_high_res(self, image_tensor):
        # Load Model
        model_url = self.config.model_url
        save_high_res_img_path = self.config.save_high_res_img
        print(model_url)
        model = hub.load(model_url)
        logger.info(f"Model loaded succesfully from {model_url}")

        high_res_img = model(image_tensor)
        high_res_img = tf.squeeze(high_res_img)
        high_res_img = high_res_img.numpy()

        high_res_img = high_res_img.astype(np.uint8)
        logger.info(f"Low resolution image components {image_tensor.shape}")
        logger.info(f"High resolution image components {high_res_img.shape}")
        
        imageio.imwrite(save_high_res_img_path, high_res_img)
        logger.info(f"High resolution image saved in  {save_high_res_img_path}")
        






    

In [13]:
start = time.time()
configuration_manager = ConfigurationManager()
image_enhance_config = configuration_manager.get_image_enhancer_config()
image_enhancer = ImageEnhacer(image_enhance_config)
low_res_image_tensor = image_enhancer.preprocess_image()
image_enhancer.low_to_high_res(low_res_image_tensor)
logger.info(f"Image processor latency: {(time.time() - start):.4f} seconds")


[2024-05-09 16:38:29,381: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-05-09 16:38:29,383: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-05-09 16:38:29,387: INFO: 2337365: Image obtained from : artifacts/data_ingestion/monkey_low_res.png:]
[2024-05-09 16:38:29,388: INFO: 2337365: Transfroming image into a tensor:]
[2024-05-09 16:38:29,393: INFO: 2337365: Returning tensor image:]
https://tfhub.dev/captain-pool/esrgan-tf2/1
[2024-05-09 16:38:33,778: INFO: load: Fingerprint not found. Saved model loading will continue.:]
[2024-05-09 16:38:33,780: INFO: load: path_and_singleprint metric could not be logged. Saved model loading will continue.:]
[2024-05-09 16:38:33,781: INFO: 2337365: Model loaded succesfully from https://tfhub.dev/captain-pool/esrgan-tf2/1:]
[2024-05-09 16:38:37,081: INFO: 2337365: Low resolution image components (1, 120, 124, 3):]
[2024-05-09 16:38:37,082: INFO: 2337365: High resolution image components (480, 496, 3):]
[2024-0